In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [2]:
import pandas as pd
import numpy as np

In [3]:
# path = "/content/gdrive/MyDrive/dataset/"
path = '../dataset new/'
path1 = '../testing/'


In [4]:
df_users_full = pd.read_csv(path + "users.csv")
df_users1 = df_users_full[["user_id","functional_unit","department"]]
df_users = df_users1[df_users1.functional_unit == "2 - ResearchAndEngineering"]
df_users = df_users[(df_users.department != "1 - Research")]
df_users = pd.DataFrame(df_users)

In [5]:
df_users = df_users.dropna(axis = 0)
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3991 entries, 8 to 17333
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          3991 non-null   object
 1   functional_unit  3991 non-null   object
 2   department       3991 non-null   object
dtypes: object(3)
memory usage: 124.7+ KB


In [6]:
df_device = pd.read_csv(path1 + "device_test.csv")

In [7]:
df_file = pd.read_csv(path1 + "file_test.csv")

In [8]:
df_logon = pd.read_csv(path1 + "logon_test.csv")

In [9]:
df_psychometric = pd.read_csv(path1 + "psychometric_test.csv")
df_psychometric = df_psychometric[['employee_name','user_id','O','C','E', 'A', 'N']]

In [10]:
df_users_clean = df_users.rename(columns= {'user_id':'user'}, inplace=False)

In [11]:
df_logon_users = pd.merge(df_logon, df_users_clean, on = 'user')
df_logon_users_clean = df_logon_users.drop(columns=['functional_unit', 'department'])


In [12]:
df_device_users = pd.merge(df_device, df_users_clean, on='user')
df_device_users_clean = df_device_users.drop(columns=['functional_unit','department'])

In [13]:
df_device_users = pd.merge(df_device, df_users_clean, on = 'user')
df_device_users_clean = df_device_users.drop(columns = ['functional_unit', 'department'])


In [14]:
df_file_users = pd.merge(df_file, df_users_clean, on = 'user')
df_file_users_clean = df_file_users.drop(columns = ['functional_unit', 'department'])


In [15]:
df_psychometric_users = pd.merge(df_psychometric, df_users, on = 'user_id')
df_psychometric_users_clean = df_psychometric_users[['employee_name', 'user_id','O', 'C', 'E','A','N']]


In [16]:
df_logon_users_clean['date'] = pd.to_datetime(df_logon_users_clean['date'])

In [17]:
df_logon_users_clean['time'] = df_logon_users_clean['date'].dt.time

In [18]:
df_user_logon = df_logon_users_clean.loc[df_logon_users_clean['activity'] == 'Logon']

In [19]:
df_user_logon['hour'] = pd.to_datetime(df_user_logon['date'], format='%H:%M').dt.hour

/tmp/ipykernel_5001/1930962928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logon['hour'] = pd.to_datetime(df_user_logon['date'], format='%H:%M').dt.hour


In [20]:
df_user_logon_stats = df_user_logon.groupby('user')['time'].agg([min,max]).reset_index()

In [21]:
df_logon_mode = df_user_logon.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [22]:
df_logon_mean = df_user_logon.groupby('user')['hour'].mean().reset_index()
df_logon_mean['hour'].dtype
df_logon_mean['hour'] = pd.to_datetime(df_logon_mean['hour'], format='%H').dt.time


In [23]:
df_user_logon_stats['mode'] = df_logon_mode['time']
df_user_logon_stats['mean'] = df_logon_mean['hour']

In [24]:
df_user_logoff = df_logon_users_clean.loc[df_logon_users_clean['activity'] == 'Logoff']

In [25]:
df_user_logoff['date'] = pd.to_datetime(df_user_logoff['date'])
df_user_logoff['time'] = df_user_logoff['date'].dt.time

/tmp/ipykernel_5001/636462469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logoff['date'] = pd.to_datetime(df_user_logoff['date'])
/tmp/ipykernel_5001/636462469.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logoff['time'] = df_user_logoff['date'].dt.time


In [26]:

df_user_logoff['hour'] = pd.to_datetime(df_user_logoff['date'], format='%H:%M').dt.hour

/tmp/ipykernel_5001/1504136962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logoff['hour'] = pd.to_datetime(df_user_logoff['date'], format='%H:%M').dt.hour


In [27]:
df_user_logoff_stats = df_user_logoff.groupby('user')['time'].agg([min,max]).reset_index()

In [29]:
# df_user_logoff.groupby('user')['time'].agg(pd.Series.mode).reset_index()

In [30]:
df_logoff_mode = df_user_logoff.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [31]:
df_logoff_mean = df_user_logoff.groupby('user')['hour'].mean().reset_index()
df_logoff_mean['hour'] = df_logoff_mean['hour'].astype(int)
df_logoff_mean['hour'] = pd.to_datetime(df_logoff_mean['hour'], format='%H').dt.time


In [32]:
df_user_logoff_stats['mode'] = df_logoff_mode['time']
df_user_logoff_stats['mean'] = df_logoff_mean['hour']

In [33]:
df_device_users_clean['time'] = pd.to_datetime(df_device_users_clean['date']).dt.time

In [34]:
df_device_conn = df_device_users_clean.loc[df_device_users_clean['activity'] == 'Connect']
df_device_disconn = df_device_users_clean.loc[df_device_users_clean['activity'] == 'Disconnect']


In [35]:
df_device_conn_stats = df_device_conn.groupby('user')['time'].agg([min, max]).reset_index()

In [36]:
df_device_conn_stats_1 = df_device_conn.groupby('user')

In [37]:
df_conn_mode = df_device_conn.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [38]:
df_device_conn['hour'] = pd.to_datetime(df_device_conn['date']).dt.hour

/tmp/ipykernel_5001/2737355688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_device_conn['hour'] = pd.to_datetime(df_device_conn['date']).dt.hour


In [39]:
df_conn_mean = df_device_conn.groupby('user')['hour'].mean().reset_index()
df_conn_mean['hour'] = df_conn_mean['hour'].astype(int)
df_conn_mean['hour'] = pd.to_datetime(df_conn_mean['hour'],format="%H").dt.time

In [40]:
df_device_conn_stats['mode'] = df_conn_mode['time']
df_device_conn_stats['mean'] = df_conn_mean['hour']

In [41]:
df_device_disconn_stats = df_device_disconn.groupby('user')['time'].agg([min,max]).reset_index()

In [42]:
df_dconn_mode = df_device_disconn.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()



In [43]:
df_device_disconn['hour'] = pd.to_datetime(df_device_disconn['date']).dt.hour


/tmp/ipykernel_5001/1753353138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_device_disconn['hour'] = pd.to_datetime(df_device_disconn['date']).dt.hour


In [44]:
df_dconn_mean = df_device_disconn.groupby('user')['hour'].mean().reset_index()
df_dconn_mean['hour'] = df_dconn_mean['hour'].astype(int)
df_dconn_mean['hour'] = pd.to_datetime(df_dconn_mean['hour'], format='%H').dt.time


In [45]:
df_dconn_mean

,user,hour
0,AJQ0376,15:00:00
1,AJR0231,12:00:00
2,AOD0066,11:00:00
3,ARH0777,12:00:00
4,BCP0247,13:00:00
5,BDS0396,12:00:00
6,BMS0057,15:00:00
7,BZK0095,11:00:00
8,CAE0080,12:00:00
9,CGH0088,12:00:00


In [46]:
df_device_disconn_stats['mode'] = df_dconn_mode['time']
df_device_disconn_stats['mean'] = df_dconn_mean['hour']

In [47]:
df_file_users_clean['date2'] = pd.to_datetime(df_file_users_clean['date']).dt.date

In [48]:
df_files_per_day = df_file_users_clean.groupby(['user', 'date2']).size().reset_index()

In [49]:
df_files_per_day.rename(columns={0:'transfers_per_day'}, inplace=True)


In [50]:
df_files_max_per_day = df_files_per_day.groupby('user')['transfers_per_day'].agg(max).reset_index()
df_files_max_per_day.rename(columns={'transfers_per_day': "max_transfers_per_user"}, inplace=True)

In [51]:
df_files_mode_per_day = df_files_per_day.groupby('user')['transfers_per_day'].agg(lambda x: x.value_counts().index[0]).reset_index()
df_files_mode_per_day.rename(columns={'transfers_per_day': "mode_transfers_per_user"}, inplace=True)


In [52]:
df_files_mode_per_day

,user,mode_transfers_per_user
0,AJQ0376,72
1,AJR0231,18
2,AOD0066,6
3,ARH0777,18
4,BCP0247,18
5,BDS0396,18
6,BMS0057,18
7,BZK0095,90
8,CAE0080,18
9,CGH0088,18


In [53]:
df_files_stats = df_files_mode_per_day

In [54]:
df_files_stats_new = pd.DataFrame()
df_files_stats_new['user'] = df_files_stats['user']
df_files_stats_new['mode_trasfers_per_user'] = df_files_stats['mode_transfers_per_user']
df_files_stats_new['max_transfers_per_user'] = df_files_max_per_day['max_transfers_per_user']


In [55]:
df_user_pc = df_logon_users_clean.groupby(['user','pc',]).agg(pc_visits_per_user_total = pd.NamedAgg(column = 'pc', aggfunc = 'count')).reset_index()
df_user_pc['count'] = df_user_pc.groupby(['user'])['pc'].transform('nunique')
df_user_pc = df_user_pc.drop(['pc', 'pc_visits_per_user_total'], axis=1)
df_user_pc = df_user_pc.drop_duplicates()

In [56]:
df_user_pc

,user,count
0,AAN0077,1
1,AAW0103,1
2,ACE0265,1
3,ACL0394,2
5,ADR0362,1
...,...,...
425,XTM0246,1
426,YJT0368,2
428,YSB0779,1
429,ZBL0379,1


In [57]:
# import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [58]:
df_user_pc_count = np.array(df_user_pc['count'])

In [59]:
df_user_pc_ct = df_user_pc_count.reshape(-1,1)

In [60]:
import pickle
model = pickle.load(open('../pkl/user_pc_ct.pkl', 'rb'))
graph_a_score = model.decision_function(df_user_pc_ct)
threat = model.predict(df_user_pc_ct)


In [61]:
#user pc
#forest = IsolationForest(bootstrap=False, contamination=0.1,max_features=1.0,
#                          max_samples='auto',n_estimators=100, n_jobs=1, random_state=None,
#                          verbose=0)
# # forest.fit(df_user_pc_ct)


In [62]:
# import pickle 
# with open('user_pc_ct.pkl','wb') as file :
#   pickle.dump(forest, file)

In [63]:
# graph_a_score = forest.decision_function(df_user_pc_ct)
# print(graph_a_score[1:10])

In [64]:
graph_result = pd.DataFrame()
graph_result['user'] = df_user_pc['user']
graph_result['ascore'] = graph_a_score
graph_result['threat']= threat
graph_result.to_csv('../output/user_pc_ct.csv', index=False)
graph_result


,user,ascore,threat
0,AAN0077,0.090123,1
1,AAW0103,0.090123,1
2,ACE0265,0.090123,1
3,ACL0394,0.000000,1
5,ADR0362,0.090123,1
...,...,...,...
425,XTM0246,0.090123,1
426,YJT0368,0.000000,1
428,YSB0779,0.090123,1
429,ZBL0379,0.090123,1


In [65]:
outliers = graph_result.loc[graph_result['ascore'] < 0]
print(outliers)

        user    ascore  threat
14   AJR0231 -0.270804      -1
35   ALC0100 -0.279173      -1
57   ARH0777 -0.176816      -1
70   BGZ0902 -0.276023      -1
114  CQS0899 -0.279173      -1
137  DNS0397 -0.082759      -1
148  EER0383 -0.278647      -1
167  FED0275 -0.279173      -1
281  LWB0078 -0.277596      -1
330  PYT0264 -0.266654      -1
401  WJP0386 -0.082759      -1
405  WMP0272 -0.279699      -1


In [66]:
def dtt2timestamp(dtt):
  time_in_sec = (dtt.hour*60 + dtt.minute) * 60 + dtt.second
  return time_in_sec

In [67]:
df_user_logon_stats_sec = df_user_logon_stats


In [68]:
min_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['min']]
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mean']]

In [69]:
df_user_logon_stats_sec['min_ts'] = min_ts
df_user_logon_stats_sec['max_ts'] = max_ts
df_user_logon_stats_sec['mode_ts'] = mode_ts
df_user_logon_stats_sec['mean_ts'] = mean_ts

In [70]:
df_user_logon_stats_sec.drop(['min','max','mode','mean'], axis=1)

,user,min_ts,max_ts,mode_ts,mean_ts
0,AAN0077,27900,29580,28800,25200
1,AAW0103,25200,26280,26100,25200
2,ACE0265,28800,30300,29700,28800
3,ACL0394,32400,60757,33360,32400
4,ADR0362,27000,28080,28080,25200
...,...,...,...,...,...
225,XTM0246,27900,29520,29460,25200
226,YJT0368,27000,68459,27900,25200
227,YSB0779,27000,28020,27900,25200
228,ZBL0379,29700,52443,30600,32400


In [71]:
df_user_logoff_stats_sec = df_user_logoff_stats


In [72]:
min_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['min']] 
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mean']]

In [73]:
df_user_logoff_stats_sec['min_ts'] = min_ts
df_user_logoff_stats_sec['max_ts'] = max_ts
df_user_logoff_stats_sec['mode_ts'] = mode_ts
df_user_logoff_stats_sec['mean_ts'] = mean_ts

In [74]:
df_user_logoff_stats_sec.drop(['min', 'max','mode','mean'], axis=1)

,user,min_ts,max_ts,mode_ts,mean_ts
0,AAN0077,60420,62040,60960,57600
1,AAW0103,58320,59340,58320,57600
2,ACE0265,57900,59400,58620,57600
3,ACL0394,34279,63000,62100,57600
4,ADR0362,60120,61200,60300,57600
...,...,...,...,...,...
225,XTM0246,60480,62100,61200,57600
226,YJT0368,59940,68732,60240,57600
227,YSB0779,60180,61200,60300,57600
228,ZBL0379,69420,71100,69600,68400


In [75]:
df_log_on_off_stats = pd.DataFrame()

df_log_on_off_stats['user'] = df_user_logon_stats_sec['user']
df_log_on_off_stats['on_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['on_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['on_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['on_mean_ts'] = df_user_logon_stats_sec['mean_ts']
df_log_on_off_stats['off_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['off_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['off_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['off_mean_ts'] = df_user_logon_stats_sec['mean_ts']

In [76]:
df_log_on_off_stats.dtypes

user           object
on_min_ts       int64
on_max_ts       int64
on_mode_ts      int64
on_mean_ts      int64
off_min_ts      int64
off_max_ts      int64
off_mode_ts     int64
off_mean_ts     int64
dtype: object

In [77]:
log_stats = df_log_on_off_stats.drop(['user'], axis = 1)
log_stats_matrix = np.matrix(log_stats)
print(log_stats_matrix)

[[27900 29580 28800 ... 29580 28800 25200]
 [25200 26280 26100 ... 26280 26100 25200]
 [28800 30300 29700 ... 30300 29700 28800]
 ...
 [27000 28020 27900 ... 28020 27900 25200]
 [29700 52443 30600 ... 52443 30600 32400]
 [25260 26760 25560 ... 26760 25560 25200]]


In [78]:
df_log_on_off_stats.columns

Index(['user', 'on_min_ts', 'on_max_ts', 'on_mode_ts', 'on_mean_ts',
       'off_min_ts', 'off_max_ts', 'off_mode_ts', 'off_mean_ts'],
      dtype='object')

In [79]:
# 

model2 = pickle.load(open('../pkl/log_stats_matrix.pkl', 'rb'))
log_ascore = model2.decision_function(log_stats_matrix)
threat = model2.predict(log_stats_matrix)


In [80]:
# # counting the values
# df22 = pd.Series(forest.predict(log_stats_matrix))
# # df22 = df22.map({1:0, -1:1})
# # print(df22.value_counts())

In [81]:
# import pickle 
# with open('log_stats_matrix.pkl','wb') as file :
#   pickle.dump(forest, file)

In [82]:
df_user_log_result = pd.DataFrame()
df_user_log_result['user'] = df_user_logoff_stats_sec['user']
df_user_log_result['ascore'] = log_ascore
df_user_log_result['threat'] = threat
df_user_log_result.to_csv('../output/user_log_result.csv', index=False)
print(df_user_log_result)


        user    ascore  threat
0    AAN0077  0.162700       1
1    AAW0103  0.098027       1
2    ACE0265  0.142234       1
3    ACL0394  0.020642       1
4    ADR0362  0.137137       1
..       ...       ...     ...
225  XTM0246  0.091681       1
226  YJT0368  0.021580       1
227  YSB0779  0.174086       1
228  ZBL0379  0.077552       1
229  ZIF0269  0.086144       1

[230 rows x 3 columns]


In [83]:
df_user_log_result.loc[df_user_log_result['ascore'] < 0]

,user,ascore,threat
13,ALC0100,-0.030278,-1
14,AMR0656,-0.013699,-1
23,BCP0247,-0.043267,-1
24,BDS0396,-0.030364,-1
25,BER0398,-0.041076,-1
26,BFM0101,-0.021495,-1
44,CAE0080,-0.018350,-1
56,CQS0899,-0.105395,-1
58,CSD0242,-0.027867,-1
65,DJS0406,-0.006024,-1


In [84]:
df_device_conn_stats_sec = df_device_conn_stats
con_min_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['min']]
con_max_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['max']]
con_mode_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['mode']]
con_mean_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['mean']]

df_device_conn_stats_sec['min_ts'] = con_min_ts
df_device_conn_stats_sec['max_ts'] = con_max_ts
df_device_conn_stats_sec['mode_ts'] = con_mode_ts
df_device_conn_stats_sec['mean_ts'] = con_mean_ts
df_device_conn_stats_sec = df_device_conn_stats_sec.drop(['min', 'max','mode','mean'], axis=1)

In [85]:
df_device_disconn_stats_sec = df_device_conn_stats
discon_min_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['min']]
discon_max_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['max']]
discon_mode_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['mode']]
discon_mean_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['mean']]

df_device_disconn_stats_sec['min_ts'] = discon_min_ts
df_device_disconn_stats_sec['max_ts'] = discon_max_ts
df_device_disconn_stats_sec['mode_ts'] = discon_mode_ts
df_device_disconn_stats_sec['mean_ts'] = discon_mean_ts
df_device_disconn_stats_sec = df_device_disconn_stats_sec.drop(['min', 'max','mode','mean'], axis=1)


In [86]:
df_device_full = pd.DataFrame()
df_device_full['user'] = df_device_conn_stats['user']

In [87]:
df_device_full['con_min_ts'] = df_device_conn_stats_sec['min_ts']
df_device_full['con_max_ts'] = df_device_conn_stats_sec['max_ts']
df_device_full['con_mode_ts'] = df_device_conn_stats_sec['mode_ts']
df_device_full['con_mean_ts'] = df_device_conn_stats_sec['mean_ts']

# disconnect stats
df_device_full['dcon_min_ts'] = df_device_disconn_stats_sec['min_ts']
df_device_full['dcon_max_ts'] = df_device_disconn_stats_sec['max_ts']
df_device_full['dcon_mode_ts'] = df_device_disconn_stats_sec['mode_ts']
df_device_full['dcon_mean_ts'] = df_device_disconn_stats_sec['mean_ts']

# files per day stats
df_device_full['file_mode'] = df_files_stats_new['mode_trasfers_per_user']
df_device_full['file_max'] = df_files_stats_new['max_transfers_per_user']

In [88]:
device_full_matrix = df_device_full.drop(['user'],axis=1)
device_params = np.matrix(device_full_matrix)
device_params[:10]

matrix([[30770, 67849, 37742, 43200, 30770, 67849, 37742, 43200,    72,
           198],
        [27100, 51362, 31751, 32400, 27100, 51362, 31751, 32400,    18,
           108],
        [28861, 53510, 29633, 36000, 28861, 53510, 29633, 36000,     6,
            24],
        [27977, 55194, 44891, 39600, 27977, 55194, 44891, 39600,    18,
            72],
        [15711, 84764, 53558, 43200, 15711, 84764, 53558, 43200,    18,
            54],
        [33007, 59723, 33631, 43200, 33007, 59723, 33631, 43200,    18,
            90],
        [29381, 61574, 40653, 43200, 29381, 61574, 40653, 43200,    18,
            72],
        [29062, 58182, 43065, 39600, 29062, 58182, 43065, 39600,    90,
           216],
        [29837, 58377, 30403, 39600, 29837, 58377, 30403, 39600,    18,
            72],
        [29633, 56751, 32112, 39600, 29633, 56751, 32112, 39600,    18,
            54]])

In [89]:
df_device_full.columns

Index(['user', 'con_min_ts', 'con_max_ts', 'con_mode_ts', 'con_mean_ts',
       'dcon_min_ts', 'dcon_max_ts', 'dcon_mode_ts', 'dcon_mean_ts',
       'file_mode', 'file_max'],
      dtype='object')

In [90]:
# # device (-user)
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)

# forest.fit(device_params)


In [91]:
# import pickle 
# with open('device_params.pkl','wb') as file :
#   pickle.dump(forest, file)

In [92]:
# 

model3 = pickle.load(open('../pkl/device_params.pkl', 'rb'))
log_ascore = model3.decision_function(device_params)
threat = model3.predict(device_params)


In [93]:
df_device_file_full_result = pd.DataFrame()

df_device_file_full_result['user'] = df_device_full['user']
df_device_file_full_result['ascore'] = log_ascore
df_device_file_full_result['threat'] = threat

df_device_file_full_result.to_csv('../output/device_file_full_result.csv', index=False)

In [94]:
df_device_file_full_result.loc[df_device_file_full_result['ascore'] < 0] 

,user,ascore,threat
4,BCP0247,-0.025266,-1
29,LJW0776,-0.032163,-1
40,SBM0063,-0.044742,-1
46,ZBL0379,-0.034616,-1


In [95]:
psychometric_matrix = df_psychometric_users_clean.drop(['user_id', 'employee_name'], axis = 1)
psychometric_params = np.matrix(psychometric_matrix)

In [96]:
# # psycho ds 
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)
# forest.fit(psychometric_params)

In [97]:
# import pickle 
# with open('psychometric_params.pkl','wb') as file :
#   pickle.dump(forest, file)

In [98]:
# 

model4 = pickle.load(open('../pkl/psychometric_params.pkl', 'rb'))
# df2=pd.DataFrame()
log_ascore = model4.decision_function(psychometric_params)
threat = model4.predict(psychometric_params)


In [99]:
df_psychometric_result = pd.DataFrame()

df_psychometric_result['user'] = df_psychometric_users_clean['user_id']
df_psychometric_result['ascore'] = log_ascore
df_psychometric_result['threat']= threat
df_psychometric_result.to_csv('../output/psychometric_result.csv', index=False)
df_psychometric_result

,user,ascore,threat
0,BMT0903,0.055850,1
1,BMT0903,0.055850,1
2,BMT0903,0.055850,1
3,BMT0903,0.055850,1
4,BMT0903,0.055850,1
...,...,...,...
863,QLC0248,0.089814,1
864,QLC0248,0.089814,1
865,QLC0248,0.089814,1
866,QLC0248,0.089814,1


In [100]:
df_psychometric_result.loc[df_psychometric_result['ascore'] < 0]

,user,ascore,threat
54,JAF0755,-0.010308,-1
55,JAF0755,-0.010308,-1
56,JAF0755,-0.010308,-1
57,JAF0755,-0.010308,-1
58,JAF0755,-0.010308,-1
...,...,...,...
809,DJH0245,-0.000770,-1
810,DJH0245,-0.000770,-1
811,DJH0245,-0.000770,-1
812,DJH0245,-0.000770,-1


In [101]:
df = pd.merge(df_log_on_off_stats, df_user_pc, on='user')

In [102]:
df_1 = pd.merge(df, df_psychometric, left_on = 'user', right_on = 'user_id')

In [103]:
df_1.head()

,user,on_min_ts,on_max_ts,on_mode_ts,on_mean_ts,off_min_ts,off_max_ts,off_mode_ts,off_mean_ts,count,employee_name,user_id,O,C,E,A,N
0,ACL0394,32400,60757,33360,32400,32400,60757,33360,32400,2,Adam Clark Lindsey,ACL0394,25,44,18,19,17
1,AGP0775,27000,28500,27900,25200,27000,28500,27900,25200,1,Alden Gary Puckett,AGP0775,39,22,36,23,28
2,AJQ0376,29820,30960,30600,28800,29820,30960,30600,28800,1,Aquila Jasper Quinn,AJQ0376,17,35,28,23,30
3,AJR0231,27060,70714,27900,32400,27060,70714,27900,32400,21,Amethyst Jillian Reed,AJR0231,43,41,13,21,30
4,ALC0100,25200,67486,26340,32400,25200,67486,26340,32400,14,Arden Leo Chase,ALC0100,11,38,14,48,28


In [104]:
df_final = df_1.drop(['employee_name', 'user_id'], axis=1)


In [105]:
df_all_parameters = df_final

In [106]:
df_all_parameters_input = df_all_parameters.drop(['user'], axis = 1)

In [107]:
print(df_all_parameters_input)

    on_min_ts  on_max_ts  on_mode_ts  on_mean_ts  off_min_ts  off_max_ts  \
0       32400      60757       33360       32400       32400       60757   
1       27000      28500       27900       25200       27000       28500   
2       29820      30960       30600       28800       29820       30960   
3       27060      70714       27900       32400       27060       70714   
4       25200      67486       26340       32400       25200       67486   
5       27060      28260       28080       25200       27060       28260   
6        1190      85217       28800       36000        1190       85217   
7       29700      31380       31320       28800       29700       31380   
8       27900      50573       28920       32400       27900       50573   
9       27000      28080       27900       25200       27000       28080   
10       5957      85207       28980       32400        5957       85207   
11      27900      50895       28380       32400       27900       50895   
12      2526

In [108]:
# # psycho logon ds
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)
# forest.fit(df_all_parameters_input)

In [109]:
# import pickle 
# with open('all_parameters_input.pkl','wb') as file :
#   pickle.dump(forest, file)

In [110]:
# 

model5 = pickle.load(open('../pkl/all_parameters_input.pkl', 'rb'))
# df2=pd.DataFrame()
log_ascore = model5.decision_function(df_all_parameters_input)
threat = model5.predict(df_all_parameters_input)


In [111]:
df_all_parameters_result = pd.DataFrame()

df_all_parameters_result['user'] = df_final['user']
df_all_parameters_result['ascore'] =log_ascore
df_all_parameters_result['threat'] = threat
print(df_all_parameters_result)
df_all_parameters_result.to_csv('../output/all_parameters_result.csv', index=False)

       user    ascore  threat
0   ACL0394 -0.003225      -1
1   AGP0775  0.135238       1
2   AJQ0376  0.095480       1
3   AJR0231  0.047736       1
4   ALC0100 -0.053868      -1
5   AWW0784  0.107327       1
6   BCP0247 -0.034859      -1
7   BMT0903  0.080123       1
8   BPH0243  0.095634       1
9   BQK0526  0.102748       1
10  CAE0080 -0.000947      -1
11  CEP0238  0.108498       1
12  CGP0271 -0.021654      -1
13  CJG0059  0.082087       1
14  CJV0236  0.130765       1
15  DJH0245  0.084275       1
16  DRH0268  0.009309       1
17  DWC0525  0.059508       1
18  EAD0651  0.109488       1
19  ETK0783  0.084516       1
20  GOG0650  0.113461       1
21  HKK0363  0.127950       1
22  HOA0074  0.125071       1
23  JAF0755  0.075980       1
24  JEC0387  0.060099       1
25  JMT0232  0.066355       1
26  JNW0252  0.040094       1
27  KCS0267  0.060643       1
28  KMJ0392  0.057290       1
29  KSM0099  0.030667       1
30  KTP0385  0.114525       1
31  KWH0391  0.025535       1
32  LDC022